# TabTransformers
In this notebook I will be deploying the TabTransformers model on the IBLI dataset

https://huggingface.co/keras-io/tab_transformer

In [82]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import seaborn as sns

from scipy.stats import f_oneway
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

from tensorflow import keras
from tensorflow import metrics
from tensorflow.keras import layers
from keras import backend as K
from tensorflow.keras.layers import Input, Dense, Embedding, Reshape, Concatenate, LayerNormalization, MultiHeadAttention, Add, Lambda, Dropout
from tensorflow.keras.models import Model
import keras_tuner as kt
from keras.metrics import MeanSquaredError, MeanAbsoluteError

In [57]:
keras.metrics.R2Score(
    class_aggregation="uniform_average", num_regressors=0, name="r2_score", dtype=None
)

<R2Score name=r2_score>

# Load and preprocess the data

In [59]:
#Load in the dataset
df = pd.read_excel("C:/Users/daanm/Documents/Universiteit Utrecht/Scriptie/cattle_df.xlsx")
df = df.drop(['cs_cs_ratio_post_cattle','buy_nr_cattle','buy_cattle', 'ratio_insured_cattle', 'n_previd_cattle'], axis=1)
df.head()

,Unnamed: 0,id,wave,afm_language,age_constant,agric_land,amh_language,educ_recoded_constant,eng_language,expend,...,number_adults,main_info_source_recoded,religion_recoded,owns_phone,household_moved,purchase_bin,why_not_purchase_recoded,know_vip,trust_vip,advise_vip
0,0,1,4,No,25,Yes,No,Never attended,No,0,...,2,Interpersonal Sources,Traditional/Wakefata,0,No,No,Lack of Awareness or Understanding,Yes,Yes,Yes
1,1,2,4,Yes,27,Yes,No,Adult Education,No,0,...,2,Interpersonal Sources,Traditional/Wakefata,0,No,No,"Financial, Practical, and Situational Constraints",Yes,Yes,Yes
2,2,3,4,Yes,29,No,No,Elementary,No,1,...,2,Professional and Organizational Sources,Christian,0,No,No,"Financial, Practical, and Situational Constraints",Yes,Yes,Yes
3,3,4,4,No,35,Yes,No,Never attended,No,0,...,2,Interpersonal Sources,Traditional/Wakefata,0,No,No,"Financial, Practical, and Situational Constraints",Yes,Yes,Yes
4,5,6,4,No,60,Yes,No,Never attended,No,0,...,2,Interpersonal Sources,Traditional/Wakefata,0,No,No,Lack of Awareness or Understanding,Yes,Yes,No


In [60]:
columns_list = df.columns.tolist()
print(columns_list)


['Unnamed: 0', 'id', 'wave', 'afm_language', 'age_constant', 'agric_land', 'amh_language', 'educ_recoded_constant', 'eng_language', 'expend', 'irrigated_land_bin', 'cs_cs_diff_post_cattle', 'number_minors', 'educ_child_recoded', 'activity_child_recoded', 'household_description', 'number_adults', 'main_info_source_recoded', 'religion_recoded', 'owns_phone', 'household_moved', 'purchase_bin', 'why_not_purchase_recoded', 'know_vip', 'trust_vip', 'advise_vip']


In [61]:
#Log+1 transforming the dataset to get a better distribution

df['cs_diff_log'] = np.log(np.abs(df['cs_cs_diff_post_cattle']) + 1)
df[['cs_cs_diff_post_cattle', 'cs_diff_log']].head()

,cs_cs_diff_post_cattle,cs_diff_log
0,-188.170624,5.242649
1,-850.367249,6.746844
2,-5219.686523,8.560384
3,-157.544266,5.066034
4,-158.881271,5.074431


In [62]:
y = df['cs_diff_log']
X = df.drop(columns=['cs_diff_log', 'cs_cs_diff_post_cattle'])

In [63]:
categorical = X.select_dtypes(include=['object','category']).columns.tolist()
numerical = X.select_dtypes(include=['int64','float64']).columns.tolist()

# Deploy the TabTransformers model
https://keras.io/examples/structured_data/tabtransformer/

In [64]:
#Encode and scale the data

label_encoders = {}
for col in categorical:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

scaler = StandardScaler()
X[numerical] = scaler.fit_transform(X[numerical])

In [65]:
# Inputs
inputs = []
embeddings = []

# Embedding categorical features
for col in categorical:
    input_cat = Input(shape=(1,), name=col)
    vocab_size = X[col].nunique()
    embed_dim = min(50, (vocab_size + 1) // 2)
    
    embedding = layers.Embedding(input_dim=vocab_size + 1, output_dim=embed_dim)(input_cat)
    embedding = layers.Reshape((embed_dim,))(embedding)
    
    inputs.append(input_cat)
    embeddings.append(embedding)

# Numerical input
input_num = Input(shape=(len(numerical),), name="numerical")
inputs.append(input_num)
embeddings.append(input_num)

# Concatenate all embeddings + numerical
x = layers.Concatenate()(embeddings)

# Project everything to a fixed dimension for transformer block
hidden_dim = 64
x = layers.Dense(hidden_dim)(x)

#Adding a feed forward network to improve the model

def feed_forward_network(hidden_dim, ff_dim, dropout_rate=0.01):
    return tf.keras.Sequential([
        layers.Dense(ff_dim, activation="relu"),   
        layers.Dropout(dropout_rate),
        layers.Dense(hidden_dim)                    
    ])

# Transformer blocks
for _ in range(2):
    x1 = layers.LayerNormalization()(x)
    attn_output = layers.MultiHeadAttention(num_heads=2, key_dim=32)(x1[:, None, :], x1[:, None, :])
    attn_output = layers.Lambda(lambda x: tf.squeeze(x, axis=1))(attn_output)
    attn_output = layers.Dropout(0.1)(attn_output)

    x2 = layers.Add()([x, attn_output])
    
    x3 = layers.LayerNormalization()(x2)
    ffn = feed_forward_network(hidden_dim=hidden_dim, ff_dim=hidden_dim*4)
    ffn_output = ffn(x3)
    x = layers.Add()([x2, ffn_output])

# Output for regression
output = layers.Dense(1)(x)

# Model
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.R2Score()])


In [66]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Construct TabTransformer input format
def make_tab_input(df, categorical, numerical):
    inputs = {col: df[col].values for col in categorical}
    inputs['numerical'] = df[numerical].values
    return inputs

X_train = make_tab_input(X_train_raw, categorical, numerical)
X_test = make_tab_input(X_test_raw, categorical, numerical)

In [67]:
X_train = {
    col: X_train_raw[col].values for col in categorical
}
X_train["numerical"] = X_train_raw[numerical].values

# Same for X_test...
X_test = {
    col: X_test_raw[col].values for col in categorical
}
X_test["numerical"] = X_train_raw[numerical].values

In [68]:
model.fit(X_train, y_train, validation_split=0.1, epochs=10, batch_size=32)


Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - loss: 7.0118 - mean_absolute_error: 1.8824 - r2_score: -1.2116 - val_loss: 2.9633 - val_mean_absolute_error: 1.2358 - val_r2_score: 0.0792
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2.8743 - mean_absolute_error: 1.2280 - r2_score: 0.0032 - val_loss: 2.8670 - val_mean_absolute_error: 1.2264 - val_r2_score: 0.1091
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2.7265 - mean_absolute_error: 1.1855 - r2_score: 0.0417 - val_loss: 2.9849 - val_mean_absolute_error: 1.1922 - val_r2_score: 0.0725
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.6955 - mean_absolute_error: 1.1685 - r2_score: 0.0642 - val_loss: 2.9488 - val_mean_absolute_error: 1.2498 - val_r2_score: 0.0837
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.7460 - mean_absolute_error: 1.1979 - r2_score: 0.0479 - val_loss: 2.8985 - val_mean_absolute_error: 1.1808 - val_r2_score: 0.0993
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms

# Hyperparameter tuning
using https://keras.io/keras_tuner/

In [83]:
def build_model(hp):
    # Hyperparameters
    hidden_dim = hp.Int('hidden_dim', min_value=32, max_value=128, step=32)
    ff_dim = hp.Int('ff_dim', min_value=128, max_value=512, step=32)
    dropout_rate = hp.Float('dropout_rate', 0.0, 0.5, step=0.1)
    num_heads = hp.Choice('num_heads', [2, 4, 8])
    learning_rate = hp.Choice('learning_rate', [1e-3, 1e-4])

    # Inputs
    inputs = []
    embeddings = []

    for col in categorical:
        input_cat = Input(shape=(1,), name=col)
        vocab_size = X[col].nunique()
        embed_dim = min(50, (vocab_size + 1) // 2)
        embedding = layers.Embedding(input_dim=vocab_size + 1, output_dim=embed_dim)(input_cat)
        embedding = layers.Reshape((embed_dim,))(embedding)
        inputs.append(input_cat)
        embeddings.append(embedding)

    input_num = Input(shape=(len(numerical),), name="numerical")
    inputs.append(input_num)
    embeddings.append(input_num)

    x = layers.Concatenate()(embeddings)
    x = layers.Dense(hidden_dim)(x)

    for _ in range(2):
        x1 = layers.LayerNormalization()(x)
        attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_dim // num_heads, dropout=dropout_rate)(
            x1[:, None, :], x1[:, None, :]
        )
        attn_output = layers.Lambda(lambda x: tf.squeeze(x, axis=1))(attn_output)
        x2 = layers.Add()([x, attn_output])

        x3 = layers.LayerNormalization()(x2)
        ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dropout(dropout_rate),
            layers.Dense(hidden_dim),
        ])
        ffn_output = ffn(x3)
        x = layers.Add()([x2, ffn_output])

    output = layers.Dense(1)(x)
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse', metrics=[
            keras.metrics.R2Score(name='r2_score'), keras.metrics.MeanSquaredError(name='mse'), keras.metrics.MeanAbsoluteError(name='mae')
        ])
    return model

## RandomSearch

Random search selects combinations of hyperparameters at random from a defined search space. Instead of trying every single possibility, it samples a fixed number of random combinations. This is a faster method and often just as effective as grid search.

In [88]:
tuner = kt.RandomSearch(
    build_model,
    objective= kt.Objective('val_r2_score', direction='max') ,
    max_trials=15,
    executions_per_trial=2,
    directory='tuner_dir_v7',
    project_name='tabtransformer_tuning_v7'
)

tuner.search(X_train, y_train, validation_split=0.2, epochs=15, batch_size=32)

Trial 15 Complete [00h 01m 17s]
val_r2_score: 0.06064307689666748

Best val_r2_score So Far: 0.08119013905525208
Total elapsed time: 00h 19m 16s


In [89]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(1)[0]

print(f"Best hidden_dim: {best_hps.get('hidden_dim')}")
print(f"Best ff_dim: {best_hps.get('ff_dim')}")

Best hidden_dim: 32
Best ff_dim: 512


C:\Users\daanm\anaconda3\envs\jupyter_env\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 106 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Random search results

In [91]:
results = []

for i, trial in enumerate(tuner.oracle.trials.values()):
    hp = trial.hyperparameters.values
    val_r2 = trial.metrics.get_best_value('val_r2_score')
    val_mse = trial.metrics.get_best_value('val_mse')
    val_mae = trial.metrics.get_best_value('val_mae')
    res = {
        'trial_id': trial.trial_id,
        **hp,
        'val_r2_score': val_r2,
        'val_mse': val_mse,
        'val_mae': val_mae
    }
    results.append(res)

results_df = pd.DataFrame(results)

In [92]:
display_cols_r2 = ['trial_id', 'hidden_dim', 'ff_dim', 'dropout_rate', 'num_heads', 'learning_rate', 'val_r2_score','val_mse','val_mae']

#Create a table sorting on the R2 score
results_table_r2 = results_df[display_cols_r2].sort_values(by='val_r2_score', ascending=False)
results_table_r2.style.background_gradient(subset=['val_r2_score'], cmap='viridis_r').background_gradient(subset=['val_r2_score'], cmap='plasma_r')

,trial_id,hidden_dim,ff_dim,dropout_rate,num_heads,learning_rate,val_r2_score,val_mse,val_mae
4,04,32,512,0.100000,4,0.000100,0.081190,2.903282,1.190108
7,07,96,448,0.200000,2,0.000100,0.071401,2.934213,1.202104
13,13,64,288,0.300000,8,0.000100,0.070992,2.935507,1.186871
10,10,128,480,0.100000,8,0.000100,0.067529,2.946448,1.195663
6,06,64,384,0.000000,8,0.000100,0.066331,2.950235,1.205410
5,05,32,160,0.400000,8,0.001000,0.065559,2.952675,1.216886
8,08,128,352,0.300000,8,0.001000,0.064602,2.955697,1.223103
1,01,96,128,0.100000,4,0.001000,0.063936,2.957802,1.201970
11,11,128,512,0.200000,2,0.000100,0.063371,2.959589,1.227762
3,03,128,320,0.100000,8,0.000100,0.061564,2.965297,1.189017


## Hyperband

Hyperband is a so-called bandit-based optimisation algorithm that speeds up hyperparameter search. It tries many configurations quickly using fewer epochs and allocating more resources to the promising candidates.

In [93]:
tuner_v2 = kt.Hyperband(
    build_model,                
    objective= kt.Objective('val_r2_score', direction='max'),   
    max_epochs=50,              
    factor=3,                   
    directory='tuner_dir',      
    project_name='tabtransformer_tuning_v4',
    overwrite=True,             
    seed=42                
)

tuner_v2.search(X_train, y_train, validation_split=0.2, epochs=15, batch_size=32)

Trial 90 Complete [00h 01m 16s]
val_r2_score: 0.08065527677536011

Best val_r2_score So Far: 0.08146995306015015
Total elapsed time: 00h 47m 32s


## Hyperband results

In [94]:
results_hyperband = []

for i, trial in enumerate(tuner_v2.oracle.trials.values()):
    hp = trial.hyperparameters.values
    val_r2 = trial.metrics.get_best_value('val_r2_score')
    val_mse = trial.metrics.get_best_value('val_mse')
    val_mae = trial.metrics.get_best_value('val_mae')
    res = {
        'trial_id': trial.trial_id,
        **hp,
        'val_r2_score': val_r2,
        'val_mse': val_mse,
        'val_mae': val_mae
    }
    results_hyperband.append(res)

results_df_hyperband = pd.DataFrame(results_hyperband)

In [95]:
display_cols_r2 = ['trial_id', 'hidden_dim', 'ff_dim', 'dropout_rate', 'num_heads', 'learning_rate', 'val_r2_score','val_mse','val_mae']

#Create a table sorting on the R2 score
results_table_r2_hyperband = results_df_hyperband[display_cols_r2].sort_values(by='val_r2_score', ascending=False)
results_table_r2_hyperband.style.background_gradient(subset=['val_r2_score'], cmap='viridis_r').background_gradient(subset=['val_r2_score'], cmap='plasma_r')

,trial_id,hidden_dim,ff_dim,dropout_rate,num_heads,learning_rate,val_r2_score,val_mse,val_mae
79,0079,96,352,0.000000,4,0.001000,0.081470,2.902398,1.200104
89,0089,128,160,0.100000,2,0.000100,0.080655,2.904972,1.201474
48,0048,32,224,0.400000,4,0.001000,0.079579,2.908372,1.201949
43,0043,32,192,0.000000,8,0.000100,0.078828,2.910748,1.207139
57,0057,96,448,0.000000,8,0.000100,0.078314,2.912369,1.183659
59,0059,64,448,0.100000,4,0.001000,0.077844,2.913855,1.198749
88,0088,32,192,0.100000,8,0.000100,0.077219,2.915832,1.182081
73,0073,64,416,0.300000,8,0.000100,0.077160,2.916018,1.220195
67,0067,96,448,0.000000,8,0.000100,0.077103,2.916197,1.190134
55,0055,64,480,0.200000,8,0.000100,0.077030,2.916429,1.182628
